In [1]:
from futu import *
import numpy as np
quote_ctx = OpenQuoteContext(host='127.0.0.1', port=11111)
code = 'US.SPY'

2022-02-06 13:40:22,525 | 1117110 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=1, host=127.0.0.1, port=11111, user_id=7465070


In [2]:
# 下月第 3 个星期五
from datetime import datetime
from dateutil.relativedelta import relativedelta, FR
first_day_of_this_month = datetime.now().replace(day=1, hour=0,minute=0,second=0,microsecond=0)
third_friday_of_next_month = first_day_of_this_month + relativedelta(months=1) + relativedelta(weekday=FR(3)) 
third_friday_of_next_month = third_friday_of_next_month.strftime('%Y-%m-%d')
third_friday_of_next_month

'2022-03-18'

In [3]:
ret, data = quote_ctx.get_option_chain(code=code, start=third_friday_of_next_month, end=third_friday_of_next_month)
if ret != RET_OK:
    print('error:', data)   
chains = data
chains.head()

,code,name,lot_size,stock_type,option_type,stock_owner,strike_time,strike_price,suspension,stock_id,index_option_type
0,US.SPY220318C100000,SPY 220318 100.00C,100,DRVT,CALL,US.SPY,2022-03-18,100.0,False,31780111,N/A
1,US.SPY220318P100000,SPY 220318 100.00P,100,DRVT,PUT,US.SPY,2022-03-18,100.0,False,31780132,N/A
2,US.SPY220318C105000,SPY 220318 105.00C,100,DRVT,CALL,US.SPY,2022-03-18,105.0,False,31780114,N/A
3,US.SPY220318P105000,SPY 220318 105.00P,100,DRVT,PUT,US.SPY,2022-03-18,105.0,False,31780148,N/A
4,US.SPY220318C110000,SPY 220318 110.00C,100,DRVT,CALL,US.SPY,2022-03-18,110.0,False,31780126,N/A


In [4]:
columns = ['code','last_price','ask_price','bid_price','strike_time','strike_price','delta', 'gamma','vega','theta', 'iv','open_interest']
def get_options(codes):
    ret, data = quote_ctx.get_market_snapshot(codes)
    if ret != RET_OK:
        print('error:', data)
        return None
    options = data
    rename_columns = {'option_strike_price':'strike_price', 'option_delta':'delta', 'option_gamma':'gamma', 
           'option_vega':'vega', 'option_theta':'theta', 'option_implied_volatility': 'iv', 
           'option_open_interest': 'open_interest'}
    options.rename(columns=rename_columns, inplace = True)
    return options[columns]

In [5]:
call_codes = chains[chains['option_type'] == 'CALL']['code'].to_list()
call_options = get_options(call_codes)
call_options.head()

,code,last_price,ask_price,bid_price,strike_time,strike_price,delta,gamma,vega,theta,iv,open_interest
0,US.SPY220318C100000,350.35,348.79,347.73,2022-03-18,100.0,0.999,0.0,0.003,-0.004,164.477,399
1,US.SPY220318C105000,363.00,343.81,342.76,2022-03-18,105.0,0.998,0.0,0.005,-0.009,164.601,2
2,US.SPY220318C110000,338.26,338.81,337.76,2022-03-18,110.0,0.998,0.0,0.006,-0.009,159.317,0
3,US.SPY220318C115000,341.93,333.82,332.76,2022-03-18,115.0,0.998,0.0,0.005,-0.007,152.547,700
4,US.SPY220318C120000,328.25,328.82,327.76,2022-03-18,120.0,0.998,0.0,0.005,-0.008,148.653,1


In [6]:
put_codes = chains[chains['option_type'] == 'PUT']['code'].to_list()
put_options = get_options(put_codes)
put_options.head()

,code,last_price,ask_price,bid_price,strike_time,strike_price,delta,gamma,vega,theta,iv,open_interest
0,US.SPY220318P100000,0.01,0.01,0.0,2022-03-18,100.0,0.0,0.0,0.001,-0.001,133.902,50315
1,US.SPY220318P105000,0.01,0.01,0.0,2022-03-18,105.0,0.0,0.0,0.001,-0.001,129.618,6864
2,US.SPY220318P110000,0.01,0.01,0.0,2022-03-18,110.0,0.0,0.0,0.001,-0.001,125.539,15421
3,US.SPY220318P115000,0.01,0.01,0.0,2022-03-18,115.0,0.0,0.0,0.001,-0.001,121.646,18227
4,US.SPY220318P120000,0.01,0.01,0.0,2022-03-18,120.0,0.0,0.0,0.001,-0.001,117.924,23495


In [7]:
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]

def find_option_by_strike(options, strike):
    return options[options['strike_price']==strike][columns].iloc[0]

def find_option(options, delta):
    # 先找到 delta 最近的期权
    nearest_delta = find_nearest(options['delta'].to_list(), delta)
    option = options[options['delta']==nearest_delta]    
    option = option.iloc[0]
    
    # 再找附近整数行权价的期权，流动性好一些
    rem = option.strike_price % 5
    strike = option.strike_price - rem if rem <3 else option.strike_price + 5 - rem
    return find_option_by_strike(options, strike)

In [8]:
call1 = find_option(call_options, 0.08)
put1 = find_option(put_options, -0.08)
call2 = find_option_by_strike(call_options, call1.strike_price+5)
put2 = find_option_by_strike(put_options, put1.strike_price-5)

In [9]:
options = [call1, call2, put1, put2]
print('price:',[call1.ask_price, call2.bid_price,put1.ask_price, put2.bid_price])
print('strike:',[x.strike_price for x in options])
print('delta:',[x.delta for x in options])
print('gamma:',[x.gamma for x in options])
print('open_interest:',[x.open_interest for x in options])
print()

max_profit = (call1.ask_price -call2.bid_price + put1.ask_price - put2.bid_price)*100
max_loos = ((call2.strike_price-call1.strike_price)-(call1.ask_price-call2.bid_price)-(put1.ask_price-put2.ask_price))*100
margin = (call2.strike_price-call1.strike_price)*100
rate_of_return = max_profit/margin*100
print('margin:', int(margin))
print('max profit:', int(max_profit))
print('max loss:', int(max_loos))
print('rate of return: %.2f%%' % rate_of_return)
print()

print('total delta: %.2f' % (((-call1.delta + call2.delta) + (-put1.delta + put2.delta))*100))
print('total gamma: %.2f' % (((-call1.gamma + call2.gamma) + (-put1.gamma + put2.gamma))*100))

price: [0.9, 0.52, 1.92, 1.64]
strike: [480.0, 485.0, 385.0, 380.0]
delta: [0.088, 0.057, -0.079, -0.068]
gamma: [0.007, 0.005, 0.002, 0.002]
open_interest: [23986, 25205, 12418, 33040]

margin: 500
max profit: 65
max loss: 436
rate of return: 13.20%

total delta: -2.00
total gamma: -0.20
